In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle

from IPython.display import display_html
from keras.models import load_model, Model
from matplotlib.colors import rgb2hex
import matplotlib.pyplot as plt
import numpy as np

from masterthesis.features.build_features import file_to_sequence
from masterthesis.utils import ATTENTION_LAYER, DATA_DIR, MODEL_DIR, RESULTS_DIR, load_split

Using TensorFlow backend.


In [3]:
def show_weighted(tokens, weights):
    cmap = plt.cm.coolwarm
    ents = ['<p style="max-width: 80ch">']
    for token, weight in zip(tokens, weights):
        col = rgb2hex(cmap(weight))
        ent = '<span style="color: %s">%s</span>' % (col, token)
        ents.append(ent)
    ents.append('</p>')
    return ' '.join(ents)

In [4]:
def tokens_in_file(filepath):
    with filepath.open() as f:
        for line in f:
            for token in line.strip().split():
                yield token

In [5]:
def file_with_attention(filepath, attention_model, w2i, maxlen=None):
    x = file_to_sequence(700, filepath, w2i)[np.newaxis, :]
    weights = attention_model.predict(x)[0]
    weights -= weights.min()
    weights /= weights.max()
    if maxlen is not None:
        weights = weights[:maxlen]
    unk_idx = w2i['__UNK__']
    tokens = (token if idx != unk_idx else 'UNK' for token, idx in zip(tokens_in_file(filepath), x[0]))
    return show_weighted(tokens, weights)

In [6]:
model_name = 'rnn_nli-25740789'
# model_name = 'rnn_nli-25717032'

In [7]:
w2i = pickle.load((MODEL_DIR / (model_name + '_model_w2i.pkl')).open('rb'))
model = load_model(str(MODEL_DIR / (model_name + '_model.h5')))

In [8]:
attention_model = Model(inputs=model.input, outputs=model.get_layer(ATTENTION_LAYER).output)

In [9]:
dev = load_split('dev')

results = pickle.load((RESULTS_DIR / (model_name + '.pkl')).open('rb'))
correct_predictions = dev.iloc[np.where(results.true == results.predictions)]
langs = ['russisk', 'vietnamesisk', 'engelsk', 'somali']
sample_indices = [correct_predictions[correct_predictions.lang == l].index[0]
                  for l in langs]
samples = correct_predictions.loc[sample_indices]
display(samples)
filenames = [DATA_DIR / 'txt' / (fn + '.txt') for fn in samples.filename]

,age,cefr,filename,gender,lang,num_tokens,split,testlevel,title,topic
183,29,B2,h0186,kvinne,russisk,551,dev,Høyere nivå,Helse og livskvalitet,helse
1014,29,A2/B1,s0180,kvinne,vietnamesisk,380,dev,Språkprøven,Røyking,helse røyking
186,51,C1,h0189,kvinne,engelsk,523,dev,Høyere nivå,Helse og livskvalitet,helse
1450,21,A2/B1,s0621,kvinne,somali,284,dev,Språkprøven,En hyggelig opplevelse,opplevelse


In [10]:
for filename in filenames:
    display_html('<h2>%s</h2>' % filename.stem, raw=True)
    html = file_with_attention(filename, attention_model, w2i, maxlen=300)
    with open(filename.stem + '.html', 'w') as f:
        print("""
<!DOCTYPE html>
<html lang="nb">
<head>
  <meta charset="utf-8">
  <title>%s</title>
  <style>
  body {
    font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
    font-size: 14px;
    line-height: 20px;
    text-align: left;
  }
  </style>
</head>
<body>
""" % filename.stem, file=f)
        print(html, file=f)
        print('</body></html>', file=f)
    display_html(html, raw=True)

h0186

UNK og livskvalitet | Undersøkelser viser at mange mennesker ikke lever sunt verden over . Folk beveger seg mindre enn før , bruker bil eller bus oftere , spiser usunn mat . Det fører til økning antall av forskjellige sykdommer blant folk . Det er et internasjonal problem . Forskere diskuterer problemet og prøver å finne noen løsninger . De UNK menneskene hvordan feil livsstil kan påvirke helse . I denne situasjonen kan legene hjelpe oss med god råd . Men det finnes ikke UNK til den . Vi er de eneste som kan forbedre situasjonen og hjelpe seg selv . Først og fremst må vi spise riktig . Mange mennesker spiser feil . De spiser mye fett mat . Spesielt usunn er " UNK " . Den er den UNK mat du kan finne . UNK må bli så UNK så mulig . UNK du mer grønnsaker og frukt - blir du UNK . UNK er også lurt å spise . Det er bedre å spise flere UNK enn du er vant til . Så kan det bli lurt å bytte ut UNK med UNK . Mange mennesker spiser UNK til middag . Det fører til forskjellige UNK . UNK viser at over femti prosent av nordmenn ikke spiser varm middag hver dag . De er vanligvis unge UNK mennesker med barn . De er så opptatt av UNK sine at de ikke finner tid til å lage mat . Det høres forferdelig ut . UNK mat er nødvendig til kroppen . Spesielt viktig er det for barn . Vi må prioritere UNK helse fordi det er de som skal drive samfunnet i framtiden . Men sunn mat er ikke den eneste UNK av høy livskvalitet og helse . UNK aktivitet er like viktig som sunn mat . Det

s0180

Oppgave 1 : UNK . | Alle tingene har to sider , ulemper og UNK . UNK er også UNK situasjonen . Det er forskjellige holdninger til røyking i Norge og UNK UNK . Først vil jeg fortelle om UNK og ulemper av røyking . Hvis vi kan røyke i riktig tid og i riktig sted , får vi ganske mye gode resultater . For UNK , når vi jobber mye og får mye stress , UNK vi en UNK . Da vi kan føle bedre fordi det har UNK i UNK og den kan hjelpe oss når vi er veldig sliten på jobben . UNK er det mange ulemper av røyking . Hvis noen UNK mye , kan de få sykdom fra røyking . Noen kan få UNK UNK etter UNK røyking . Noen blir død av den grunnen . UNK mye er ikke godt for helsa . En ulempe av røyking til vil jeg UNK om som er mange UNK fra røyking . De er bil UNK , UNK . Når mannen kjører bil og UNK , vil mannen finne en UNK , Da etter kan ikke mannen UNK . Så man kan få bil ulike . Noen liker å røyke mens de ligger seg på UNK . De sover og glemmer å kaste UNK . Det er mange UNK UNK på denne grunnen . Andre vil jeg UNK om det er mye forskjellige UNK til røyking i Norge og Vietnam . UNK UNK UNK mye . De UNK tidligere enn UNK . UNK kan røyke når de er barn UNK 13 årene . Men det er forbudt for barna som er under 18 år i Vietnam . Norsk kvinner UNK UNK mest i verden . I Vietnam er det få kvinner som kan røyke r . De

h0189

Oppgave A | Hvert menneske kan bidra til å bedre sin egen helse og dette kan UNK på flere måter . Samtidig er det viktig å huske at et menneske er også et UNK og at det finnes ikke noen UNK for alle . UNK helse for de alle fleste vil være et resultat av en UNK av hvordan man lever i dag og hvor UNK UNK ligger . Å endre UNK har visst seg å være vanskelig for mange . Et eksempel på dette er UNK holdning til bruk av UNK og alkohol . Selv om det er UNK UNK at både UNK og UNK reduserer bade UNK og UNK velger mange å fortsette å bruke de daglig . Hvis vi tar UNK i vårt vestlige samfunnet er det å ikke røyke sentralt i et hvert forsøk for med å bedre egen helse . Et annet viktig UNK er hva vi spiser og ikke minst hvor mye . En diet med et større UNK av frukt og grønnsaker er en av de mest UNK av UNK i alle UNK UNK . UNK innen UNK har i økende grad UNK om på UNK av mosjon i forholdet til både fysiske og psykiske helse . Det er kosthold og mosjon som bør stå mest sentralt hvis en skal ta vare på egen helse . Men det er også viktig å UNK over de sosiale UNK en har , arbeidsledigheten over lang tid for eksempel er ikke en situasjon som bidrar til bedre UNK over UNK Den UNK UNK av UNK UNK i befolkningen har lenge vært brukt som et mål for UNK . Hvis en mann kunne forvente å leve til 50 år i et UNK land mens den tilsvarende UNK UNK var UNK år i Norge var UNK at en

s0621

Om en hyggelig opplevelse | Jeg hadde en hyggelig og UNK da jeg var liten , fordi faren min var rik og han hadde stor butikk som selger nesten alle ting som man trenger . I tilleg var han medlem av en stor UNK som lager møbler , også han hadde en gård som han UNK forskjellig grønnsaker og det var noen som jobbet der men han går av og til for å se hvordan det er . Han UNK de som arbeider på UNK . Dessuten bodde vi en stort rekke hus og vi hadde UNK som UNK barna , UNK , lager og UNK hjemme . moren min jobbet ikke , hun var UNK men hun kunne skrive og lese så hjelper hun oss på UNK . UNK min søsken vi går på skolen fra mandag til fredag og det var moren min som kjørte oss til skolen . I tilleg vi går på privat skole på bare engelsk fordi denne skolen vi har bare UNK på alle UNK derfor vi gikk på engelsk privat skole det var mange mennesker som kunne ikke dette , fordi de hadde ikke råd til å betale men heldigvis vi hadde på grunn av faren oss var rik . Dessuten har vi også lært UNK ( religion ) hjemme og det var en mann som kom til oss for å undervise oss . Det var en fantastisk og hyggelig opplevelse som jeg skal aldri gleme i forhold til hvor jeg har kommet men har UNK det er vanlig opplevelse . Det betyr at hjemmelandet mit er ikke så rik som norge er , derfor jeg sier at jeg har UNK en hyggelig og godt opplevelse .